In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Input
from tensorflow.keras.utils import to_categorical

In [2]:
def create_var(x,value = 10):
    locals()[x] = value
    print(locals()[x])

In [3]:
pose = ("P1R1",)
out_frame_no = 10
for pose_no in (pose):
    input1 = np.load("C:/Users/lalas/OneDrive - dfki.de/PresSim/Data/FinalData/3DPose/"+pose_no+"_pose.npy")
    input2 = np.load("C:/Users/lalas/OneDrive - dfki.de/PresSim/Data/FinalData/SimulatedPressure/"+pose_no+"_sim.npy")
    output = np.load("C:/Users/lalas/OneDrive - dfki.de/PresSim/Data/FinalData/PressueMap/"+pose_no+"_Sen.npy") 
    buffer1 = np.empty([10,17,3])
    buffer2 = np.empty([10,80,28])
    buffer3 = np.empty([1,80,28])
    in_1 = np.empty([(input1.shape[0]-10),10,17,3,1])
    in_2 = np.empty([(input1.shape[0]-10),10,80,28,1])
    out = np.empty([int((input1.shape[0])-10),1,80,28,1])
    np.save("gt11.npy", out)
    for x in range(0,(input1.shape[0]-10), 1):
        buffer3[0] =  output[x+out_frame_no]
        for y in range (10):
            buffer1[y] =  input1[x+y]
            buffer2[y] =  input2[x+y]
        in_1[x] = tf.expand_dims(buffer1, 3)
        in_2[x] = tf.expand_dims(buffer2, 3)
        out[x] = tf.expand_dims(buffer3, 3)
    print(in_1.shape,in_2.shape,out.shape)

(13404, 10, 17, 3, 1) (13404, 10, 80, 28, 1) (13404, 1, 80, 28, 1)


In [4]:
print(np.max(out))
print(np.min(out))

2833.0
0.0


In [5]:
#10 frames 3dcnn
droprate = 0.3
a = layers.Input(shape=(10, 17, 3, 1), name="Input_a")
output1 = layers.Reshape((10, 17, 3))(a)
output1 = layers.Conv2D(filters = 128, kernel_size = (5,8),  activation='relu')(output1) #no padding, critical for shaping the tensor
# don't change this layer's kernel without adjusting the pooling and upsampling
output1= layers.MaxPooling2D(pool_size=(2, 1))(output1)
output1= layers.UpSampling2D(size=(1, 2))(output1)
output1= layers.Dropout(droprate)(output1)
output1= layers.BatchNormalization()(output1)
output1= layers.Conv2D(28, kernel_size=(5, 5), padding = 'same', activation='relu')(output1)
output1= layers.MaxPooling2D(pool_size=(2, 1))(output1)
output1= layers.Dropout(droprate)(output1)
output1= layers.BatchNormalization()(output1)
output1= layers.UpSampling2D(size=(1, 4))(output1)
output1 = layers.Reshape((80,28,1))(output1)
model1 = tensorflow.keras.Model(inputs=a,outputs=[output1], name="model_1")
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_a (InputLayer)         [(None, 10, 17, 3, 1)]    0         
_________________________________________________________________
reshape (Reshape)            (None, 10, 17, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 10, 128)        15488     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 10, 128)        0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 3, 20, 128)        0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 20, 128)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 3, 20, 128)        512 

In [6]:
b = layers.Input(shape=(10,80, 28, 1), name="Input_b")
output2 = layers.Conv3D(filters = 64, kernel_size = (3,3,3), padding = 'same', activation='relu')(b)
output2 = layers.MaxPooling3D(pool_size=(2, 1, 1))(output2)
output2 = layers.Dropout(droprate)(output2)
output2 = layers.BatchNormalization()(output2)
output2 = layers.Conv3D(128, kernel_size=(3, 3, 3), padding='same',activation='relu')(output2)
output2 = layers.MaxPooling3D(pool_size=(2, 1, 1))(output2)
output2 = layers.Dropout(droprate)(output2)
output2 = layers.BatchNormalization()(output2)
output2 = layers.Conv3D(5, kernel_size=(2, 3, 3), padding='same',activation='relu')(output2)
output2 = layers.MaxPooling3D(pool_size=(2, 1, 1))(output2)
#output2 = layers.UpSampling3D(size=(5, 4, 4))(output2)
output2 = layers.Conv3D(1, kernel_size=(1, 3, 3), padding='same',activation='relu')(output2)
output2 = layers.Reshape((80,28,1))(output2)
model2 = tensorflow.keras.Model(inputs=b,outputs=[output2], name="model_2")
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_b (InputLayer)         [(None, 10, 80, 28, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 80, 28, 64)    1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 80, 28, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 80, 28, 64)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 80, 28, 64)     256       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 80, 28, 128)    221312    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 80, 28, 128)    0   

In [7]:
output3 = layers.Concatenate()([output1,output2])
output3 = layers.Conv2D(filters = 32, kernel_size = (2,2), activation='relu', padding = 'same')(output3)
output3 = layers.Conv2D(filters = 8, kernel_size = (2,2), activation='relu', padding = 'same')(output3)
output3 = layers.Conv2D(filters = 1, kernel_size = (1,1), activation='relu', padding = 'same')(output3)
output3 = layers.Reshape((1,80,28))(output3)

In [8]:

model = tensorflow.keras.Model(inputs=[a,b],outputs=[output3], name="regressor_model")
model.compile(loss='mse',optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.001), metrics=["mae"])
model.summary()

Model: "regressor_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_a (InputLayer)            [(None, 10, 17, 3, 1 0                                            
__________________________________________________________________________________________________
Input_b (InputLayer)            [(None, 10, 80, 28,  0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 10, 17, 3)    0           Input_a[0][0]                    
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 10, 80, 28, 6 1792        Input_b[0][0]                    
____________________________________________________________________________________

In [ ]:
history = model.fit([in_1,in_2],out,
            batch_size=128,
            epochs=40,
            verbose=1,
            validation_split=0.3
                   )

Epoch 1/40
74/74 [==============================] - 31s 298ms/step - loss: 5510.7329 - mae: 13.7977 - val_loss: 6479.5664 - val_mae: 62.5152
Epoch 2/40
74/74 [==============================] - 20s 272ms/step - loss: 5372.4102 - mae: 17.3864 - val_loss: 3730.5479 - val_mae: 13.3771
Epoch 3/40
42/74 [================>.............] - ETA: 7s - loss: 5260.2524 - mae: 17.0329

In [ ]:
pred = model.predict([in_1,in_2]) 

In [ ]:
pred.shape
np.save("pd11.npy", pred)
np.save("gt11.npy", out)
out.shape

In [ ]:
ind = 13100
gt = np.squeeze(out[ind,0])
pd = np.squeeze(pred[ind])

In [ ]:
print(gt.shape)
print(pd.shape)

In [ ]:
plt.figure(figsize=(6, 3.2))
ax = plt.subplot(1,2,1)
ax.set_title('gt')
plt.imshow(gt)
ax.set_aspect('equal')
ax = plt.subplot(1,2,2)
ax.set_title('pd')
plt.imshow(pd)
ax.set_aspect('equal')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 3.2))
ax = fig.add_subplot(111)
plt.axis('off')
plt.imshow(gt)
ax.set_aspect('equal')
#cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
#cax.get_xaxis().set_visible(False)
#cax.get_yaxis().set_visible(False)
ax.set(xlabel=None)
#cax.patch.set_alpha(0)
#cax.set_frame_on(False)
plt.savefig('gt.png')
fig = plt.figure(figsize=(6, 3.2))
ax = fig.add_subplot(111)
plt.axis('off')
plt.imshow(pd)
ax.set_aspect('equal')
#cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
#cax.get_xaxis().set_visible(False)
#cax.get_yaxis().set_visible(False)
#cax.patch.set_alpha(0)
#cax.set_frame_on(False)
plt.savefig('pd.png')

In [ ]:
orb = cv2.ORB_create()

# Now detect the keypoints and compute
# the descriptors for the query image
# and train image
queryKeypoints, queryDescriptors = orb.detectAndCompute(cv2.imread('gt.png'),None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(cv2.imread('pd.png'),None)

# Initialize the Matcher for matching
# the keypoints and then match the
# keypoints
matcher = cv2.BFMatcher()
matches = matcher.match(queryDescriptors,trainDescriptors)

# draw the matches to the final image
# containing both the images the drawMatches()
# function takes both images and keypoints
# and outputs the matched query image with
# its train image
final_img = cv2.drawMatches(cv2.imread('gt.png'), queryKeypoints,
cv2.imread('pd.png'), trainKeypoints, matches[:20],None)

final_img = cv2.resize(final_img, (1000,650))

# Show the final image
cv2.imwrite('keypointa.png',final_img)

In [ ]:
import sys
from PIL import Image
import glob, os

def compare_pixels(png1, png2):
  pixels1 = list(png1.getdata())
  pixels2 = list(png2.getdata())
  if len(pixels1) != len(pixels2):
    return -1 # different pixel count
  match = 0
  mismatch = 0
  for i in range(0, len(pixels1)):
    if pixels1[i] == pixels2[i]:
      match = match + 1
    else:
      mismatch = mismatch + 1
  return match/(match+mismatch)
blur = Image.open('pd.png', mode='r', formats=None)
org = Image.open('gt.png', mode='r', formats=None)
print(compare_pixels(org,blur))